# "Blog - migrate to quarto"
> "following migration guide from Hamel Hussain"
- show_tags: true
- toc: true
- branch: master
- badges: false
- comments: true
- categories: [nbdev, quarto, fastpages]
- image: images/icons/fastai.png

# Inspiration

Hamel just announced that [fastpages will be discontinued](https://forums.fast.ai/t/fastpages-deprecating-fastpages-in-favor-of-quarto/99095) as nbdev+quarto is now a valid option to provide a blogging platform.
He has written a [migration guide](https://nbdev.fast.ai/tutorials/blogging.html) for that. 

This is my walkthrough.


# Walkthrough

## install quarto

```bash
$ sudo apt install quarto
[sudo] password for guillaume:
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
quarto is already the newest version (1.1.189).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
```

will have to see what will happen on platforms where I don't have admin rights.

Here Quarto is already present because I use it for nbdev2.


## create repo `blog`

I create `blog` repo on github.

And I can now get it locally: `git clone https://github.com/castorfou/blog.git` (I am from office, only https is accepted)

## create a quarto blog

```bash
cd ~/git/blog
quarto create-project --type website:blog .
quarto install extension quarto-ext/video
```

Issue here with 

```bash
$ quarto install extension quarto-ext/video
Sending fatal alert BadCertificate
ERROR: TypeError: error sending request for url (https://github.com/quarto-ext/video/archive/refs/heads/main.tar.gz): error trying to connect: invalid peer certificate contents: invalid peer certificate: UnknownIssuer
```

## copy former blog content - notebooks and markdown files

Your new repo will have a `posts/` directory. 

This is where you will copy all of your notebook and markdown posts from fastpages. 

```bash
cp -r ../guillaume_blog/_notebooks/* posts/
cp -r ../guillaume_blog/_posts/* posts/
```

## copy former blog content - images

```bash
cp -r ../guillaume_blog/images/* posts
cp -r ../guillaume_blog/images/copied_from_nb/* posts
```

## migrate posts to quarto

```bash
conda activate dataset_tools #this is an env with nbdev installed
nbdev_migrate --path posts
```

Issue here with

```bash
$ nbdev_migrate --path posts
Traceback (most recent call last):
  File "/home/guillaume/miniconda/envs/dataset_tools/lib/python3.9/sre_parse.py", line 1051, in parse_template
    this = chr(ESCAPES[this][1])
KeyError: '\\x'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/guillaume/miniconda/envs/dataset_tools/bin/nbdev_migrate", line 8, in <module>
    sys.exit(nbdev_migrate())
  File "/home/guillaume/miniconda/envs/dataset_tools/lib/python3.9/site-packages/fastcore/script.py", line 119, in _f
    return tfunc(**merge(args, args_from_prog(func, xtra)))
  File "/home/guillaume/miniconda/envs/dataset_tools/lib/python3.9/site-packages/nbdev/migrate.py", line 168, in nbdev_migrate
    if f.name.endswith('.md'): migrate_md(f)
  File "/home/guillaume/miniconda/envs/dataset_tools/lib/python3.9/site-packages/nbdev/migrate.py", line 153, in migrate_md
    txt = fp_md_fm(path)
  File "/home/guillaume/miniconda/envs/dataset_tools/lib/python3.9/site-packages/nbdev/migrate.py", line 89, in fp_md_fm
    return _re_fm_md.sub(_dict2fm(fm), md)
  File "/home/guillaume/miniconda/envs/dataset_tools/lib/python3.9/re.py", line 327, in _subx
    template = _compile_repl(template, pattern)
  File "/home/guillaume/miniconda/envs/dataset_tools/lib/python3.9/re.py", line 318, in _compile_repl
    return sre_parse.parse_template(repl, pattern)
  File "/home/guillaume/miniconda/envs/dataset_tools/lib/python3.9/sre_parse.py", line 1054, in parse_template
    raise s.error('bad escape %s' % this, len(this))
re.error: bad escape \x at position 248 (line 10, column 23)
```